In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from transformers import pipeline
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
df=pd.read_csv('/content/lyrics_10k.csv')
def clean_text(text, max_length=578):
    cleaned_text = ' '.join(text.split())
    #print(cleaned
    cleaned_text = cleaned_text.lower()
    cleaned_text = cleaned_text.translate(str.maketrans("", "", string.punctuation))
    #print(cleaned_text)
    cleaned_text = ''.join([i for i in cleaned_text if not i.isdigit()])
    #print(cleaned_text)
    cleaned_text = ''.join([i for i in cleaned_text if i.isalpha() or i.isspace()])
    #print(cleaned_text)
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(cleaned_text)
    cleaned_text = ' '.join([word for word in word_tokens if word.lower() not in stop_words])
    #print(cleaned_text)
    if len(cleaned_text) > max_length:
        cleaned_text = cleaned_text[:max_length]
    else:
        cleaned_text = cleaned_text.ljust(max_length)
    #print(cleaned_text)
    return cleaned_text


In [ ]:
df['cleaned_lyrics'] = df['lyrics'].apply(clean_text)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load the model and tokenizer
model_name = 'bhadresh-savani/distilbert-base-uncased-emotion'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Initialize the emotion detection pipeline with truncation
emotion_detection = pipeline(
    'text-classification',
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    truncation=True
)

# Apply emotion detection to the cleaned lyrics
df['emotion'] = df['cleaned_lyrics'].apply(lambda x: emotion_detection(x)[0]['label'])
print(df[['cleaned_lyrics', 'emotion']])


                                         cleaned_lyrics  emotion
0     better home lyrics feel body breathing feel he...     fear
1     illusions lyrics people tell need help people ...    anger
2     touching ground lyrics swear touched saw heave...     love
3     nothing stopping lyrics nothing stopping stopp...      joy
4     shall believe lyrics come lay hands even lie s...  sadness
...                                                 ...      ...
8669  neon forest lyrics im sliding like lizard bell...     love
8670  harambe lyrics mafia billboard hitmakers yeah ...      joy
8671  lyrics seems like dreams like ive always could...      joy
8672  translations türkçe español selena gomez fun f...    anger
8673  translations türkçe português english english ...      joy

[8674 rows x 2 columns]
